# Python `continue` statement: practical patterns, comparisons, and tests

This notebook demonstrates common and advanced usages of `continue` across for/while loops, data cleaning, nested loops, for-else behavior, and micro-benchmarks, with tests you can run inline.

In [ ]:
# 1) Import and Environment Check
import sys, timeit, io
print(sys.version)
assert sys.version_info >= (3, 8), "Python >= 3.8 required"
print("env-ok")

In [ ]:
# 2) Basic For Loop with continue
out = []
for i in range(10):
    if i % 2 == 1:
        continue
    out.append(i)
print("even:", out)
out

In [ ]:
# 3) Using continue in while Loops
i = 0
seen = []
while i < 10:
    i += 1  # increment early to avoid infinite loop
    if i % 3 == 0:
        continue
    seen.append(i)
print(seen)
seen

In [ ]:
# 4) Skipping Invalid Data with try/except + continue
data = ["10", "20", "x", "30", "-1", "oops"]
parsed = []
for s in data:
    try:
        parsed.append(int(s))
    except ValueError:
        print("skip:", s)
        continue
print(parsed)
parsed

In [ ]:
# 5) Nested Loops: continue in Inner vs Outer Loop
matrix = [
    [1, 2, 3],
    [4, 5, 6],
]
# Inner skip: skip values equal to 5
res_inner = []
for r in matrix:
    row = []
    for v in r:
        if v == 5:
            continue
        row.append(v)
    res_inner.append(row)

# Outer skip via flag: skip entire row if it contains a 2
res_outer = []
for r in matrix:
    skip_row = False
    for v in r:
        if v == 2:
            skip_row = True
            break
    if skip_row:
        continue
    res_outer.append(list(r))
res_inner, res_outer

In [ ]:
# 6) for-else Behavior with continue
out = []
for i in range(5):
    if i == 2:
        continue  # does NOT prevent else
    out.append(i)
else:
    out.append("else-ran")
print(out)

out2 = []
for i in range(5):
    if i == 2:
        break  # prevents else
    out2.append(i)
else:
    out2.append("else-ran")
print(out2)
(out, out2)

In [ ]:
# 7) continue vs break vs pass
vals = []
for i in range(4):
    if i == 1:
        continue
    if i == 3:
        break
    if i == 2:
        pass  # no-op
    vals.append(i)
vals

In [ ]:
# 8) File-like Processing: Skip Blank and Comment Lines
text = io.StringIO("""
# header

alpha

# comment
beta
""")
cleaned = []
for line in text:
    s = line.strip()
    if not s or s.startswith('#'):
        continue
    cleaned.append(s)
print(cleaned)
cleaned

In [ ]:
# 9) Micro-benchmark: continue vs if Guards (timeit)
setup = """
N = 10000
"""
stmt_continue = """
s = 0
for i in range(N):
    if i % 2:
        continue
    s += i
"""
stmt_if_guard = """
s = 0
for i in range(N):
    if not (i % 2):
        s += i
"""
reps = 5
res1 = timeit.repeat(stmt_continue, setup=setup, repeat=reps, number=200)
res2 = timeit.repeat(stmt_if_guard, setup=setup, repeat=reps, number=200)
print("continue min/median:", min(res1), sorted(res1)[len(res1)//2])
print("if-guard min/median:", min(res2), sorted(res2)[len(res2)//2])

In [ ]:
# 10) Unit Tests for continue Examples (pytest style)

def filter_even(xs):
    out = []
    for x in xs:
        if x % 2:
            continue
        out.append(x)
    return out


def parse_ints(xs):
    out = []
    for s in xs:
        try:
            out.append(int(s))
        except ValueError:
            continue
    return out

# Tiny asserts (do not require pytest runner)
assert filter_even(list(range(6))) == [0, 2, 4]
assert parse_ints(["1", "x", "2"]) == [1, 2]
print("tests-ok")

In [ ]:
# 11) Pitfall Demo: continue inside finally (SyntaxError)
code = """
for i in range(3):
    try:
        pass
    finally:
        continue
"""
try:
    compile(code, "<string>", "exec")
except SyntaxError as e:
    print("caught:", e.__class__.__name__, str(e).splitlines()[0])
else:
    print("no-syntax-error?!")